### 注意事项
- 需要提前安装好ollama: https://ollama.com/
- 在cmd运行：ollama run gemma2，即可安装开源大模型gemma2
- 支持各类强大的开源大模型：
    - internlm2
    - codegeex4
    - glm4
    - llama3
    - qwen2
    - deepseek-coder-v2
    - phi3

##### 让开源大模型安全地润色我们的论文

我们今天来讨论如何让安全地让大模型帮我们改论文~处理机密数据

在处理敏感信息的时候比如：
- 将要发布的论文
- 企业核心数据
如果使用闭源大模型的API，可能会有数据泄露的风险。（多数大模型公司都会保护用户隐私，但是也不可避免存在风险）
如果担心这类风险，我们可以用开源大模型，直接部署到本地的电脑或者服务器中，离线运行，直接用这些模型自身生成回答，避免传输到网络中。
直观地解释，就是我们将这个模型文件保存到了电脑里，我们的问题都是会经过这个文件进行处理，而不用上传到网络。

唯一的问题就是让开源大模型做回答，对于自己的电脑的配置要求比较高。显卡内存必须超过8G才能流畅运行目前10亿参数以下的最强模型类似glm4,谷歌的gemma2等。

目前最强的开源大模型Qwen-72B足以比肩国内最强AI比如deepseek,智谱等，但是这么大的模型，普通的消费级笔记本电脑是运行不了的。
如果有服务器，可以将这个大模型部署到自己的服务器中使用这个模型来分析敏感数据。
这个大模型足够聪明，能够胜任非常多复杂的任务。

能在自己电脑上运行的大模型只有10B以下的**qwen2,internlm2,gemma2,glm4**等，这些模型事实上已经足够强大。

我们可以参考前几期的框架，构建一个用本地大模型分析论文的智能体，让大模型帮我们总结论文、给论文打分以及改论文等等。
分别对应代码中的summarize、rating和polish，效果如图所示。

需要注意的是，如果论文字数过长，这几个模型可能表现不会特别好，因此建议限制一下输入字数。
仅有qwen2和glm4支持128K的上下文。

agent-101项目是个人维护的项目，近期的智能体较为复杂，有一些文件的资料在逐渐上传中~欢迎持续关注



![大模型排行-2407](images/1720751759154.png)

### ollama 开源大模型分析论文

In [6]:
from openai import OpenAI
from functools import wraps  
from super_agent_tools.code_interpreter import *
from super_agent_tools.json_tool import *
from super_agent_tools.pdf_reader import *

class OllamaEssayAgent:
    """
    目标：使用开源大模型分析论文，包括总结、打分、优化
    优点：免费，支持中文，不存在机密信息泄露风险
    缺陷：对电脑配置要求高，模型能力不如最强GPT-4o(但是已经足够聪明)
    """
    def __init__(self, model):
        self.model = model
        self.system_prompt = "你必须尽可能详细回答我的问题。"
        self.conversations = [{"role": "system", "content": self.system_prompt}]
        # 总结论文的提示词
        self.summarize_prompt = self.get_prompt("prompts/ollama_essay_agent/summarize_prompt.txt")
        # 论文打分的提示词
        self.rating_prompt = self.get_prompt("prompts/ollama_essay_agent/rating_prompt.txt")
        # 论文优化的提示词
        self.polish_prompt = self.get_prompt("prompts/ollama_essay_agent/polish_prompt.txt")
        
    def get_prompt(self, path):
        with open(path, 'r', encoding="utf-8") as file:
            prompt = file.read()
        return prompt

    def get_ollama_yield(self, messages, model):
        client = OpenAI(
            base_url = 'http://localhost:11434/v1',
            api_key='ollama', 
        )

        response = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
        )
        for chunk in response:
            yield chunk.choices[0].delta.content

    def get_pdf_text(self, pdf_path):
        """
        从PDF文件中提取文本
        """
        return extract_clean_text_from_pdf(pdf_path, method='pdfminer')
    
    # 分析论文：总结、打分、优化
    def analyze_essay(self, pdf_path, limit=4e4, analyze_type="summarize"):
        text = self.get_pdf_text(pdf_path)[:int(limit)]
        print(f"PDF提取字数:{len(text)}")
        # 判断论文分析的类型
        if analyze_type == "summarize":
            prompt = self.summarize_prompt.format(text=text)
        elif analyze_type == "rating":
            prompt = self.rating_prompt.format(text=text)
        elif analyze_type == "polish":
            prompt = self.polish_prompt.format(text=text)
            
        conversations = [{"role": "system", "content": self.system_prompt}]
        conversations.append({"role": "user", "content": prompt})

        ans = ""
        for char in self.get_ollama_yield(conversations, self.model):
            print(char, end="", flush=True)
            ans += char
        print()
        return ans


In [7]:
# ollama聊天测试
question = "通俗易懂地解释Transformer模型每一层的原理。"

model = "gemma2"
model = "internlm2"
model = "codegeex4"
model = "glm4" # 128K
model = "qwen2" # 128K

ollama_agent = OllamaEssayAgent(model)
ollama_agent.conversations.append({"role": "user", "content": question})

ans = ""
for char in ollama_agent.get_ollama_yield(ollama_agent.conversations, model):
    print(char, end="", flush=True)
    ans += char

当然可以！为了帮助理解，我们从头开始梳理一个Transformer模型中的每一层功能。

### 1. **嵌入层（Embedding Layer）**

- **作用**：将输入数据转换为可以供神经网络处理的形式，通常是文本序列的情况下，将单词映射到高维空间的向量。
- **原理**：
  - 对每个单词进行分词得到相应的标记或索引（如在BERT模型中使用的词表）。
  - 将这些索引转换为其对应的嵌入向量。嵌入层通常会有一个权重矩阵，该矩阵决定了每个索引与高维空间中对应位置的向量。

### 2. **位置编码（Positional Encoding Layer）**

- **作用**：帮助模型理解输入序列中单词的位置信息。
- **原理**：
  - 对于每一个位置i，生成一个特定的位置编码向量。这个编码可以包含时间信息和相对距离信息。
  - 常见的是使用数学函数生成，例如sin和cos函数应用在不同频率的指数上，这样可以捕获序列中元素的时间顺序信息。

### 3. **自注意力机制（Self-Attention Layer）**

- **作用**：允许模型考虑输入序列中的每个词与所有其他词之间的相关性。
- **原理**：
  - 输入由一个查询(Q)，键(K)和值(V)组成，分别代表对输入序列中每个单词的特征提取。Q、K、V都从原始输入映射到不同的向量空间中。
  - 模型计算每一对词之间的相似性（注意力权重）并通过这些权重加权平均值V得到新的输出表示。公式为：\[Attention(Q,K,V) = softmax(\frac{Q \cdot K^T}{\sqrt{d_k}}) \cdot V\]，其中\(d_k\)是键的维度。
  - 这个过程允许模型聚焦于序列中的不同部分，这对于处理自然语言任务非常有效。

### 4. **多头自注意力（Multi-Head Attention）**

- **作用**：通过并行计算多个注意力机制来提高模型的表达能力。
- **原理**：
  - 将输入分割成多个较小的子空间（头部），每个子空间内使用一个独立的自注意力机制进行处理。这允许模型从不同的“关注视角”捕获信息，增强对上下文的理解和任务泛化能力。

### 5. **前馈神经网络（Feed-Forward Neural Network）**

- 

In [8]:
# 测试论文文件
pdf_path = r'E:\SuperAgent\智能体2.0开发-深度\files\TREE SEARCH FOR LANGUAGE MODEL AGENTS.pdf'

model = "gemma2"
model = "internlm2"
model = "glm4"
model = "qwen2"

ollama_agent = OllamaEssayAgent(model)
# 总结论文
summary = ollama_agent.analyze_essay(pdf_path, analyze_type="summarize")
print("="*50)
# 为论文评分
rating = ollama_agent.analyze_essay(pdf_path, analyze_type="rating")

PDF提取字数:40000
本文主要介绍了在预训练语言模型（Language Model, LM）的基础上，引入搜索算法来改善其在现实世界的web任务中的表现。文章的结构可以概括为：

**1. 预训练语言模型面临的挑战**

首先，文中阐述了利用语言模型作为基础进行自动推理和决策时遇到的主要问题：采样不确定性。当从文本分布中随机抽样时生成动作，首次产生的行动可能并非在环境中最佳的行动。此外，在实时、复杂的web环境中，预训练的语言模型有时会面临失败情况，尤其是无法正确理解网页布局或用户意图等。

**2. 介绍搜索策略**

为了克服上述挑战，作者引入了一种基于搜索的最佳优先树搜索算法。此方法通过构建多条可能的动作轨迹来扩展行动空间，并评估每条轨迹的潜在结果。具体来说：

- **Best-First Tree Search**: 在每个动作决策点上，利用预训练的语言模型（LM）预测不同选项的结果，并采用深度优先策略探索可能的路径。
  
- **Feedback Integration**: 实行每一步后，根据环境反馈来调整和优化未来步骤的预测。

- **Trajectory Evaluation**: 利用模拟器在虚拟环境中执行这些轨迹评估，以选择最有效的路径实现目标。

**3. 论文的主要发现**

该方法在多个VisWebArena基准测试中显著提升了LM代理的表现：

- **高成功率**: 搜索策略能帮助LM代理克服初次生成的不理想行动，并通过探索找到更高效的执行顺序来完成任务。
  
- **性能提升**: 与未采用搜索的基线相比，具有搜索能力的LM模型在多个复杂web任务中的成功率大幅提升。

**4. 实现限制**

文章也探讨了实现中可能遇到的问题：

- **时间延迟**: 搜索过程需要更多推理计算时间来评估不同的动作路径，从而增加了任务完成的整体时间。
  
- **不可逆操作风险**: 某些网页操作（如在电子商务网站上提交订单）可能会改变页面状态，并且难以恢复。

**结论**

总而言之，本文提出了一个有效的方法，在预训练的语言模型代理执行复杂web任务时引入搜索策略，以此显著提升其性能并提高完成任务的成功率。该研究为构建能够自主进行规划、推理和实际操作的智能代理奠定了基础。尽管面临一些实现限制，但作者强调了在未来工作中利用更高效算

In [9]:
# 优化论文
polish = ollama_agent.analyze_essay(pdf_path, analyze_type="polish")

PDF提取字数:40000
论文《Inference-time Search Enhances Language Model Agents for Web Tasks》在提出一种新的搜索算法以提高语言模型代理在实际网页任务中的能力时，确实有其创新和亮点，并且为当前领域提供了一种有前景的方法。然而，在评价其合理性以及潜在优化空间时，我们可能可以从几个角度进行考量：

### 1. 方法论与实验设计的详细度
- **优化建议**：
   - 增加更多定量和定性的方法比较：可以添加更多的基线模型对比，比如引入无搜索、只依赖LM预测的结果进行比较。这能够更直观地展示所提方法的优势。
   - 对于评估指标的解释和选择应更加详细。例如，如果使用了成功完成任务的比例、平均执行时间等作为评估指标，请在论文中详细阐述为什么选择了这些指标，以及它们如何反映模型的有效性和效率。

### 2. 实验场景与数据集的局限性
- **优化建议**：
   - 对于使用的数据集（如VisualWebArena），应进一步讨论其适用范围和可能的局限性。特别是当数据集规模、类型或复杂度在现实世界中有较大差异时，如何迁移所提方法的有效性和效能。
   - 需要更多地探讨数据标注的质量和一致性问题，并评估这些问题对最终结果的影响。

### 3. 模型选择与搜索策略的优化
- **优化建议**：
   - 调整或引入更先进的LM模型。在现有的研究中，通常会使用GPT等模型进行实验。考虑更多地探索其他类型的语言模型（如Transformer、BERT等），以评估其对搜索算法性能的影响。
   - 研究不同的搜索策略的组合和效果，比如深度优先搜索、广度优先搜索、A*搜索等是否能够提高任务执行效率或成功率。

### 4. 搜索过程中的细节优化
- **优化建议**：
   - 详细讨论并对比了在搜索过程中参数（如c=20）的选择对结果的影响。可以提供更广泛的参数探索，或者通过元学习方法自动调整这些参数。
   - 对于如何减少搜索带来的额外计算开销和时间成本进行更深入的研究。比如，利用缓存机制存储部分搜索路径的结果，或在模型训练阶段优化相关策略以减轻运行时的负担。

### 5. 可持续性和泛化能力
- **优化建议**：
   - 讨论方法在长时间执行和大规模部署中的可持续性问题，包括对能源消耗、环境影